### IMPORT LIBRARY

In [1]:
import os
import glob
from dotenv import load_dotenv
import json
import gradio as gr
from openai import OpenAI

In [2]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser

import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go

### DEFINE CONSTANT

In [3]:
MODEL_GPT = "gpt-4o"
MODEL_QWEN3b = 'qwen2.5:3b'
MODEL_QWEN7b = 'qwen2.5'
MODEL_LLAMA  = 'llama3.2'
db_name = "..\\vector_db"

In [4]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
openai = OpenAI()

### HANDLE KNOWLEDGE-BASE

In [5]:
import fitz

def pdf_to_md(pdf_path, output_md_path):
    """
    Converts a PDF file to a Markdown (.md) file.

    Args:
        pdf_path (str): Path to the input PDF file.
        output_md_path (str): Path to the output Markdown file.
    """
    header = ("CHƯƠNG", "1.", "2.", "3.", "4.", "5.", "6.", "7.", "8.", "9.")#, "a)", "b)", "c)", "d)", "đ)", "e)")
    try:
        pdf_document = fitz.open(pdf_path)

        with open(output_md_path, 'w', encoding='utf-8') as md_file:
            for page_number in range(len(pdf_document)):
                page = pdf_document.load_page(page_number)  # Load page
                text = page.get_text("text")  # Extract text
                lines = text.split('\n')

                paragraph = []
                prev_line = "start"
                for i in range(len(lines)):
                    line = lines[i].strip()
                    if line == "" and prev_line == "":
                        continue
                    elif line == "" and prev_line != "":
                        paragraph.append(line)
                    elif line.startswith(header):
                        paragraph.append("")
                        paragraph.append(line)
                    else:
                        paragraph.append(line)
                    prev_line = line

                text = "\n".join(paragraph)

                md_file.write(text)
                md_file.write("\n\n---\n\n")  

        print(f"Markdown file created at: {output_md_path}")

    except Exception as e:
        print(f"An error occurred: {e}")

pdf_path = "../knowledge-base/document/qcdt_2023_upload.pdf" 
output_md_path = "../knowledge-base/document/qcdt_2023_upload.md"  

pdf_to_md(pdf_path, output_md_path)


An error occurred: module 'fitz' has no attribute 'open'


In [6]:
folders = glob.glob("../knowledge-base/*")

text_loader_kwargs = {'encoding': 'utf-8'}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

### CHUNK DOCUMENTS

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

### VECTOR EMBEDDING AND STORE

In [8]:
embeddings = OpenAIEmbeddings()

In [9]:
import pandas as pd
df = pd.DataFrame([d.page_content for d in chunks], columns=["text"])

In [10]:
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

In [ ]:
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore: {vectorstore._collection.count()} documents")

In [ ]:
collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

In [16]:
result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
doc_types = [metadata['doc_type'] for metadata in result['metadatas']]
colors = [['blue', 'green'][['document', 'question answer'].index(t)] for t in doc_types]

In [17]:
tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

### BUILD INVERTED INDEXING AND TF-IDF

In [11]:
import string
from collections import defaultdict
import math

def preprocess(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text.split()

def build_inverted_index(documents):
    inverted_index = defaultdict(list)
    for doc_id, doc in enumerate(documents):
        words = preprocess(doc)
        for word in set(words):  
            inverted_index[word].append(doc_id)
    return inverted_index

def compute_tf_idf(documents, inverted_index):
    tf_idf = defaultdict(lambda: defaultdict(float))
    total_documents = len(documents)
    
    for doc_id, doc in enumerate(documents):
        words = preprocess(doc)
        word_count = len(words)
        word_freq = defaultdict(int)
        for word in words:
            word_freq[word] += 1
        
        for word, count in word_freq.items():
            tf = count / word_count
            idf = math.log(total_documents / (1 + len(inverted_index[word])))  
            tf_idf[doc_id][word] = tf * idf
    return tf_idf

def search(query, inverted_index, tf_idf, documents):
    query_words = preprocess(query)
    doc_scores = defaultdict(float)
    
    for word in query_words:
        if word in inverted_index:
            for doc_id in inverted_index[word]:
                doc_scores[doc_id] += tf_idf[doc_id].get(word, 0)
    
    sorted_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)
    
    return [(doc_id, documents[doc_id], score) for doc_id, score in sorted_docs]


In [32]:
import time

time_start = time.time()

documents_search = [doc.page_content for doc in chunks]
inverted_index = build_inverted_index(documents_search)
tf_idf = compute_tf_idf(documents_search, inverted_index)
result = search('Hãy cho tôi biết về thông tin tính điểm GPA', inverted_index, tf_idf, documents_search)
time_end = time.time()
time_execute = time_end - time_start
time_execute

0.06786823272705078

### AI AGENTS

In [18]:
from langchain_core.tools import tool
from langchain.agents import initialize_agent

ranking = { 0.0: "Yếu", 2.0: "Trung bình", 2.5: "Khá", 3.2: "Giỏi", 3.6: "Xuất sắc"}

@tool
def get_ranking(grade):
    """
    Trả về xếp loại dựa vào số điểm CPA

    Args:
        grade (float): Số điểm CPA của sinh viên
    """
    grade = grade.replace("'", "\"")

    grade = json.loads(grade).get('grade')
    grade = float(grade)

    if grade > 4 or grade < 0:
        return "Số điểm không hợp lệ trên thang điểm 4"

    for key in sorted(ranking.keys(), reverse= True):
        if grade >= key:
            return ranking[key]

    return "Xuất sắc" 


### LLMs

In [19]:
from langchain_openai import ChatOpenAI
llm_llama = ChatOpenAI(
    api_key="ollama",
    model= MODEL_LLAMA,
    base_url="http://localhost:11434/v1",
)

llm_gpt = ChatOpenAI(
    model = MODEL_GPT,
    temperature= 0.7
)

tools = [get_ranking]

llm_gpt = llm_gpt.bind_tools(tools)
agent = initialize_agent(tools, llm_gpt, agent_type="zero-shot-react-description", handle_parsing_errors=True, verbose=True)


def handle_get_ranking_tool_call(tool_call):
    arguments = tool_call.get('args')
    grade = arguments.get('grade')
    hint = f"Lấy xếp loại của sinh viên dựa vào điểm số CPA cung cấp: {grade}"
    ranking = agent.run(hint)
    return ranking


C:\Users\HoMinhNhat\AppData\Local\Temp\ipykernel_10924\78109912.py:16: LangChainDeprecationWarning:

LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/



In [20]:
retriever = vectorstore.as_retriever()

In [34]:
time_start = time.time()
result = retriever.invoke('Hãy cho tôi biết về thông tin tính điểm GPA')
time_end = time.time()

In [35]:
result

[Document(metadata={'doc_type': 'question answer', 'source': '..\\knowledge-base\\question answer\\qa_output.md'}, page_content='Điều kiện được xét học bổng loại A là gì: *: GPA từ 3.6, điểm rèn luyện từ 90.\n\nTheo định hướng của Ban Giám đốc Đại học với sinh viên NĂM THỨ TƯ, bạn nên làm gì?!: Hoàn thiện kỹ năng mềm và chuyên môn, ngoại ngữ, Đồ án tốt nghiệp, Tìm việc/ học bổng ĐH/Quỹ đầu tư…\n\nNộp hồ sơ chế độ chính sách miễn giảm học phí tại đâu?: Ban Công tác sinh viên\n\nĐâu là quy trình đánh giá kết quả điểm rèn luyện của sinh viên?: Thu thập, xác nhận tham gia hoạt động – Đánh giá kết quả điểm rèn luyện - Sử dụng kết quả điểm rèn luyện - Tự đánh giá kết quả điểm rèn luyện\n\nChính sách hỗ trợ vay vốn ngân hàng cho sinh viên là gì: *: Mức vay tối đa: 4 triệu đồng/ tháng x 10 tháng/năm học.: Mức vay tối đa: 4 triệu đồng/ tháng x 10 tháng/năm học.\n\nQuy đổi theo thang 10, điểm A là bao nhiêu điểm?: 8.5-9.4\n\nĐiểm rèn luyện được xếp loại theo cách nào dưới đây?: Xuất 

In [23]:
result[:5]

[Document(metadata={'doc_type': 'question answer', 'source': '..\\knowledge-base\\question answer\\qa_output.md'}, page_content='Học bổng Trần Đại Nghĩa dành cho sinh viên có hoàn cảnh kinh tế khó khăn, có nghị lực vươn lên trong cuộc sống có 2 mức học bổng, đó là những mức nào?: 50% và 100%\n\nHãy chọn các học bổng cho sinh viên của Đại học Bách khoa Hà Nội:: Học bổng Khuyến khích học tập: Học bổng Trần Đại Nghĩa: Học bổng Gắn kết Quê hương\n\nThời gian đào tạo chuẩn của bậc Cử nhân là bao lâu?: 4 năm\n\nQuy tắc sắp xếp thời gian Khoa học: Toàn diện: Hợp lý: Nổi bật trọng điểm: Dành khoảng thời gian trống\n\nNếu một sinh viên bị ốm nặng và phải nhập viện trong thời gian thi cuối kỳ, sinh viên đó nên làm gì?: Báo với giảng viên và làm thủ tục xin hoãn thi\n\nQuy định về trang phục đối với sinh viên tại Đại học Bách Khoa Hà Nội ?: Sinh viên không cần mặc đồng phục mọi lúc nhưng trang phục phù phải hợp với môi trường học tập; đồng thời yêu cầu sinh viên đeo thẻ sinh viên/học viên 

In [24]:
system_message ="Bạn là một chuyên gia tư vấn về quy chế đào tạo \
                cho một đại học ở Việt Nam, Đại học Bách khoa Hà Nội."

In [25]:
parser = StrOutputParser()

In [26]:
from functools import reduce

def chat_word_search(message, history):
    inverted_index = build_inverted_index(documents_search)
    tf_idf = compute_tf_idf(documents_search, inverted_index)

    results = search(message, inverted_index, tf_idf, documents_search)

    docs = [doc for doc_id, doc, score in results]
    doc_st = reduce(lambda a, b: a + " \n" + b, docs[0:5])

    qa_prompt = f"""
                Bạn hãy đưa ra các câu trả lời bằng Tiếng Việt.
                Bạn cần tư vấn chính xác những gì bạn biết và trả lời 
                thành thật những nội dung trong tài liệu bạn được cung cấp phía dưới. 
                Khi nội dung được hỏi không có thông tin trong tài liệu được 
                cung cấp, hãy nói không có thông tin trong tài liệu.

                Tài liệu: {doc_st}

                Câu hỏi: {message}
                """
    
    messages =  [{"role": "system", "content": system_message}] + \
                history[-2:] + \
                [{"role": "user", "content": qa_prompt}]

    llm_result = llm_gpt.invoke(messages)

    if llm_result.tool_calls:
        for tool_call in llm_result.tool_calls:
            if tool_call.get('name') == 'get_ranking':
                result = handle_get_ranking_tool_call(llm_result.tool_calls[0])
        return result
    
    parsed_result = parser.invoke(llm_result)
    return parsed_result

In [27]:
def chat_vector_search(message, history):
    context = retriever.invoke(message)
    print(context)
    qa_prompt = f"""
                Bạn hãy đưa ra các câu trả lời bằng Tiếng Việt.
                Bạn cần tư vấn chính xác những gì bạn biết và trả lời 
                thành thật những nội dung trong tài liệu bạn được cung cấp phía dưới. 
                Khi nội dung được hỏi không có thông tin trong tài liệu được 
                cung cấp, hãy nói không có thông tin trong tài liệu.

                Tài liệu: {context}

                Câu hỏi: {message}
                """
    
    if history:
        messages = [{"role": "system", "content": system_message}] + \
                history[-2:] + \
                [{"role": "user", "content": qa_prompt}]
    else:
        messages = [{"role": "system", "content": system_message}] + \
                [{"role": "user", "content": qa_prompt}]

    llm_result = llm_gpt.invoke(messages)

    if llm_result.tool_calls:
        for tool_call in llm_result.tool_calls:
            if tool_call.get('name') == 'get_ranking':
                result = handle_get_ranking_tool_call(llm_result.tool_calls[0])
        return result
    
    parsed_result = parser.invoke(llm_result)
    return parsed_result
    

In [38]:
view = gr.ChatInterface(chat_word_search, type="messages").launch(share = True)

* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://6d18194182995d44d1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


C:\Users\HoMinhNhat\AppData\Local\Temp\ipykernel_10924\78109912.py:23: LangChainDeprecationWarning:

The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.





> Entering new AgentExecutor chain...

Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I should use the get_ranking function to determine the ranking based on the provided CPA score of 3.7. 

Action: get_ranking
Action Input: { "grade": 3.7 }
Observation: Xuất sắc
Thought:I now know the final answer.

Final Answer: Với điểm số CPA 3.7, xếp loại của sinh viên là "Xuất sắc".

> Finished chain.


### EVALUATE

In [29]:
with open('../test/augment_QA.json', mode= 'r') as file:
    datas = json.load(file)


In [30]:
total_point = 0
def chat_evaluate(message, question):
    eval_system_prompt ="Bạn là một giảng viên sẽ đưa ra câu hỏi và đánh \
                        giá câu trả lời của sinh viên về quy chế đào tạo"
    
    answer_evaluate_prompt = f"""
                Bạn sẽ đánh giá câu trả lời của sinh viên đưa ra dưới đây
                so với đáp án được cung cấp với câu hỏi tương ứng.
                Nếu câu trả lời giống với đáp án và trả lời bằng tiếng việt,
                bạn sẽ cộng thêm 1 điểm vào tổng điểm cho sinh viên.
                
                Câu hỏi: {question['question']}

                Câu trả lời đúng: {question['answer']}

                Câu trả lời của sinh viên: {message}
                """
    
    messages =  [{"role": "system", "content": eval_system_prompt}] + \
                [{"role": "user", "content": answer_evaluate_prompt}]

    llm_result = llm_llama.invoke(messages)
    parsed_result = parser.invoke(llm_result)
    return parsed_result
    

In [39]:
result = chat_word_search("Sinh viên có thể liên hệ Ban Công tác Sinh viên qua email nào?", [])
chat_evaluate(result, datas[0])

'Tôi sẽ đánh giá câu trả lời của sinh viên như sau:\n\nCâu trả lời của sinh viên đã giống với đáp án được cung cấp. Tuy nhiên, họ đã viết lại câu trả lời bằng tiếng Việt hoàn toàn giống với đáp án, không có bất kỳ sự thay đổi nào.\n\nDo đó, tôi sẽ cộng thêm 1 điểm vào tổng điểm cho sinh viên vì câu trả lời của họ chính xác và đầy đủ.'

In [40]:
result

'Sinh viên có thể liên hệ Ban Công tác Sinh viên qua email: ctsv@hust.edu.vn.'